# Fine-tuning a model with the Trainer Wandb Chinese

## 安裝套件

In [1]:
!pip install datasets evaluate transformers[sentencepiece] --quiet # 安裝 datasets, evaluate, transformers 和 sentencepiece 庫
!pip install accelerate -U --quiet # 升級 accelerate 庫
!pip install wandb --quiet # 安裝 wandb 庫

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.2 MB/s eta 0:00:00


In [12]:
import wandb # 導入 Weights and Biases 庫
wandb.login() # 登錄 Weights and Biases 賬戶

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [13]:
from datasets import load_dataset # 導入 load_dataset 函數，用於加載數據集
from transformers import AutoTokenizer, DataCollatorWithPadding # 導入自動標記器和數據收集器

raw_datasets = load_dataset("sepidmnorozy/Chinese_sentiment") # 加載中文情感分析數據集
checkpoint = "google-bert/bert-base-chinese" # 設置 BERT 中文模型的檢查點
tokenizer = AutoTokenizer.from_pretrained(checkpoint) # 使用自動標記器從檢查點加載標記器


def tokenize_function(example): # 定義一個函數來對數據進行標記化處理
    return tokenizer(example["text"], truncation=True)  # 使用標記器對每個示例的 "text" 字段進行標記化，並進行截斷


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True) # 使用 map 方法對數據集進行標記化處理，並設置 batched=True 以啟用批處理
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # 創建一個數據收集器，用於將樣本批次化並進行填充

AttributeError: partially initialized module 'datasets' has no attribute 'utils' (most likely due to a circular import)

### 定義`Trainer`的參數
我們定義我們之前的第一步`Trainer`是定義一個類別，其中包含將用於訓練和評估的`TrainingArguments`所有超參數。為求簡單，這邊`Trainer`提供的參數是儲存訓練模型的目錄以及訓練的輪數。對於其餘所有內容，保留預設值，這對於基本的微調應該非常有效。

In [10]:
from transformers import TrainingArguments # 導入 TrainingArguments，用於設置訓練過程的參數

training_args = TrainingArguments("test-trainer", num_train_epochs=3, report_to="wandb") # 設置訓練參數

使用`AutoModelForSequenceClassification`有兩個標籤的類別

In [15]:
from transformers import AutoModelForSequenceClassification # 導入 AutoModelForSequenceClassification，用於加載預訓練的序列分類模型

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2) # 從預訓練檢查點加載 BERT 模型，並將其配置為進行序列分類任務

NameError: name 'checkpoint' is not defined

### 說明
在實例化這個預訓練模型後會收到一個警告。這是因為BERT沒有在對句子對進行分類的任務上進行預訓練，因此預訓練模型的頭部被丟棄了(bert-base-uncase)，並且加入了一個適合序列分類的新頭部。表示有些權重未被使用（對應於被丟棄的預訓練頭部的那些）以及一些其他權重是隨機初始化的（新頭部）

### 定義`Trainer`
有了模型，就可以傳入建構的所有對象來定義一個訓練器模型、`training_args`、訓練和驗證資料集、`data_collator`和分詞器

In [16]:
from transformers import Trainer  # 導入 Trainer 類，用於訓練和評估模型

trainer = Trainer( # 創建一個 Trainer 實例
    model,# 指定要訓練的模型
    training_args, # 訓練參數，定義了訓練過程的設置
    train_dataset=tokenized_datasets["train"], # 訓練數據集
    eval_dataset=tokenized_datasets["validation"], # 驗證數據集
    data_collator=data_collator, # 數據收集器，用於批處理和填充
    tokenizer=tokenizer, # 標記器，用於處理輸入文本
)

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
partially initialized module 'datasets' has no attribute 'utils' (most likely due to a circular import)

### 說明
這樣傳遞分詞器時，訓練器使用的預設`data_collator`將是之前定義的`DataCollatorWithPadding`，因此可以跳過`data_collator=data_collator`這一行

### 開始訓練
呼叫`Trainer`中的`train()`方法

In [17]:
trainer.train()  # 啟動模型訓練過程

NameError: name 'trainer' is not defined

In [18]:
trainer.save_model("./testmodel") # 將訓練好的模型保存到 "./testmodel" 目錄

NameError: name 'trainer' is not defined

In [19]:
from transformers import pipeline# 從 transformers 模塊中導入 pipeline

classifier = pipeline("sentiment-analysis", model="./testmodel") # 創建一個情感分析的流水線，並使用指定的訓練好的模型 "./testmodel"


OSError: Incorrect path_or_model_id: './testmodel'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [20]:
classifier(
    [
        "真的很棒",  # 要進行情感分析的第一個文本，表示正面情感
        "只會來這一次了", # 要進行情感分析的第二個文本，表示負面情感
    ]
)

NameError: name 'classifier' is not defined

In [21]:
model.config.id2label # 獲取模型配置中的 id2label 屬性

NameError: name 'model' is not defined